In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from torch_geometric.loader import DataLoader
from CustomDataset import CustomDataset

%config InlineBackend.figure_format = 'png2x'
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 'stix' is similar to Times New Roman
# If you want to use LaTeX to render all text (for more consistency)
# plt.rcParams['text.usetex'] = True

plt.rcParams['axes.linewidth'] = 0.5    # Axis line width
plt.rcParams['lines.linewidth'] = 1    # Line width
plt.rcParams['xtick.major.width'] = 0.5  # Set x major tick width to 2 points
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['lines.markersize'] = 4
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [36]:
# Number of time steps
nt = 12

# Number of testing data
test_size = 200

# Number of branches
num_branches = 1991

# Power flow limit
file_path = f'../../data/IEEE_Case1354/branch_params/PF_max_category1.csv'
PF_max_category1 = pd.read_csv(file_path, header=None, index_col=None).to_numpy().astype(bool).flatten()
file_path = f'../../data/IEEE_Case1354/branch_params/PF_max_category2.csv'
PF_max_category2 = pd.read_csv(file_path, header=None, index_col=None).to_numpy().astype(bool).flatten()
file_path = f'../../data/IEEE_Case1354/branch_params/PF_max_category3.csv'
PF_max_category3 = pd.read_csv(file_path, header=None, index_col=None).to_numpy().astype(bool).flatten()

# Set line flow threshold
epsilon = 0.85

PF_max1 = 500
PF_max2 = 1000
PF_max3 = 2000

PF_threshold = np.zeros((num_branches, nt))
PF_threshold[PF_max_category1, :] = epsilon * PF_max1
PF_threshold[PF_max_category2, :] = epsilon * PF_max2
PF_threshold[PF_max_category3, :] = epsilon * PF_max3

In [37]:
# Matrix to store results
branch_flow_true = []
branch_flow_pred = []

over_load_true = []
over_load_pred = []

# Read PF file
for i in range(test_size):
    # Ground truth
    file_path = f'../../res/IEEE_Case1354/model_evaluation/PF/true_corr/true_corr_{i+1}.csv'
    temp = pd.read_csv(file_path, header=None, index_col=None).to_numpy()
    branch_flow_true.append(temp.copy())

    # Prediction
    file_path = f'../../res/IEEE_Case1354/model_evaluation/PF/pred_corr/pred_corr_{i+1}.csv'
    temp = pd.read_csv(file_path, header=None, index_col=None).to_numpy()
    branch_flow_pred.append(temp.copy())

# Convert to numpy array
branch_flow_true = np.abs(np.array(branch_flow_true))
branch_flow_pred = np.abs(np.array(branch_flow_pred))

over_load_true = branch_flow_true >= PF_threshold
over_load_pred = branch_flow_pred >= PF_threshold

In [ ]:
# Probability of heavying loading
prob_true = over_load_true.mean(axis=0)
prob_pred = over_load_pred.mean(axis=0)

# Get sorted branch index according to mean probability of ground truth
sorted_idx = np.argsort(prob_true.T.mean(axis=0))

# Select six important branches
num_top_branches = 8
top_branch_idx = sorted_idx[-num_top_branches:]

# Sort top branch index
# top_branch_idx = np.sort(top_branch_idx)

top_branch_idx

## Failure mode 1 probability: 

$$\mathcal{P}^I_o(t) = p\left( \Gamma_t \right)$$

In [ ]:
columns = [f'T={i+1}' for i in range(nt)]
pd.DataFrame(prob_true[top_branch_idx], columns=columns)

In [ ]:
pd.DataFrame(prob_pred[top_branch_idx], columns=columns)

## Failure mode 2 probability:

$$\mathcal{P}^{I+}_o(t) = p\left( \bigcup_{t'=t+1}^{t+\Delta T} \Gamma_{t'} \, \vert \, \Gamma_t \right)$$

In [ ]:
# Number of time steps
T = 12
delta_T = 2

num_sig_branches = 8

cond_prob_true = np.zeros((num_sig_branches, T))
cond_prob_pred = np.zeros((num_sig_branches, T))

# Calculate conditional probability ground truth
for i in range(num_sig_branches):
    for j in range(T-delta_T):
        idx = over_load_true[:, sorted_idx[-(i+1)], j] == True
        num = (over_load_true[idx, sorted_idx[-(i+1)], j+1:j+1+delta_T] == True).sum(axis=1)
        cond_prob_true[i, j] = (num >= 1).sum() / idx.sum()

# Calculate conditional probability prediction
for i in range(num_sig_branches):
    for j in range(T-delta_T):
        idx = over_load_pred[:, sorted_idx[-(i+1)], j] == True
        num = (over_load_pred[idx, sorted_idx[-(i+1)], j+1:j+1+delta_T] == True).sum(axis=1)
        cond_prob_pred[i, j] = (num >= 1).sum() / idx.sum()

In [ ]:
pd.DataFrame(cond_prob_true[:, :-delta_T])

In [ ]:
pd.DataFrame(cond_prob_pred[:, :-delta_T])

## Faliure mode 3 probability

$$\mathcal{P}^{II}_o(t) = \left[ p\left( \Gamma_t^j \, \vert \, \Gamma_t^i \right) \right]_{i, j}$$

In [78]:
# List to store results
mode3_prob_true = np.zeros((nt, num_sig_branches, num_sig_branches))
mode3_prob_pred = np.zeros((nt, num_sig_branches, num_sig_branches))

# Calculate conditional probability ground truth
for i in range(nt):
    for j in range(num_sig_branches):
        for k in range(num_sig_branches):
            idx = over_load_true[:, sorted_idx[-(j+1)], i] == True  
            mode3_prob_true[i, j, k] = (over_load_true[idx, sorted_idx[-(k+1)], i] == True).sum() / idx.sum()

# Calculate conditional probability prediction
for i in range(nt):
    for j in range(num_sig_branches):
        for k in range(num_sig_branches):
            idx = over_load_pred[:, sorted_idx[-(j+1)], i] == True  
            mode3_prob_pred[i, j, k] = (over_load_pred[idx, sorted_idx[-(k+1)], i] == True).sum() / idx.sum()

## Failure mode 4 probability

$$\mathcal{P}^{II+}_o(t) = \left[ p\left(\bigcup_{t'=t+1}^{t+\Delta T} \Gamma_{t'}^j \, \vert \, \Gamma_t^i \right) \right]_{i, j}$$

## Risk analysis

#### Failure model 1

In [41]:
# Define cost function
Co = 1 # Cost of overloading, $5/MW

# Calculate the amount of overloading 
branch_overflow_true = np.abs(branch_flow_true - PF_threshold)
branch_overflow_pred = np.abs(branch_flow_pred - PF_threshold)

## Risk associated with overloading at time t
cost_true = branch_overflow_true * Co
cost_pred = branch_overflow_pred * Co
risk_true = cost_true.mean(axis=0).sum(axis=0)
risk_pred = cost_pred.mean(axis=0).sum(axis=0)

#### Failure mode 2

In [42]:
cond_risk_true = np.zeros((num_branches, T))
cond_risk_pred = np.zeros((num_branches, T))

# Calculate conditional probability ground truth
for i in range(num_branches):
    for j in range(T-delta_T):
        idx = over_load_true[:, sorted_idx[-(i+1)], j] == True
        cond_cost_true = branch_overflow_true[:, sorted_idx[-(i+1)], j+1:j+1+delta_T] * Co
        cond_risk_true[i, j] = cond_cost_true.mean(axis=0).sum()
cond_risk_true = cond_risk_true.sum(axis=0)

# Calculate conditional probability prediction
for i in range(num_branches):
    for j in range(T-delta_T):
        idx = over_load_true[:, sorted_idx[-(i+1)], j] == True
        cond_cost_pred = branch_overflow_pred[:, sorted_idx[-(i+1)], j+1:j+1+delta_T] * Co
        cond_risk_pred[i, j] = cond_cost_pred.mean(axis=0).sum()
cond_risk_pred = cond_risk_pred.sum(axis=0)

#### Overall risk

In [43]:
overall_risk_true = risk_true + cond_risk_true
overall_risk_pred = risk_pred + cond_risk_pred

In [ ]:
# Plot overall risk
x = np.arange(1, 11)

fig = plt.figure(figsize=(3, 1.75))
ax = fig.add_subplot(111)
ax.plot(x, overall_risk_true[:-delta_T]/1e6, color='red', marker='^', label='MILP')
ax.plot(x, overall_risk_pred[:-delta_T]/1e6, color='blue', marker='^', label='GNN')
ax.set_xlabel('Time step $t$ (h)')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_ylim((0, 3))
ax.set_ylabel('$\Re_o(t)$ ($\$10^6$)')
ax.legend(ncols=2, frameon=False)

plt.tight_layout()
plt.show()
plt.close('all')
